## Quickstart: $J/\Psi \rightarrow \gamma \pi^0 \pi^0$ decay

In this quickstart example it is shown how to use ComPWA via the python interface. The workflow is:

1. Create a model for the decay.
2. Generate a Monte Carlo data sample (hit & miss) using this model. 
3. Perform a fit on the data sample using the Minuit2 interface.
4. Visualize the data and the fit result!

Let's go!

First we `import` the necessary expert system module parts

In [1]:
from pycompwa.expertsystem.ui.system_control import (
    StateTransitionManager, InteractionTypes)
from pycompwa.expertsystem.amplitude.helicitydecay import (
    HelicityAmplitudeGeneratorXML)
from pycompwa.expertsystem.amplitude.canonicaldecay import (
   CanonicalAmplitudeGeneratorXML)
from pycompwa.expertsystem.state.particle import load_particle_list_from_xml
load_particle_list_from_xml("particle_list.xml")
# just a little function to print the intermediate states
def print_intermediate_states(solutions):
    from pycompwa.expertsystem.topology.graph import (
        get_intermediate_state_edges)
    print("intermediate states:")
    intermediate_states = set()
    for g in solutions:
        edge_id = get_intermediate_state_edges(g)[0]
        intermediate_states.add(g.edge_props[edge_id]['@Name'])
    print(intermediate_states)

### Step 1: Creating the Decay Model

#### 1.1. Define problem set

First we define the boundary conditions of our physics problem, such as

- initial state
- final state
- formalism type
- ...

Pass all of that information to the `StateTransitionManager`, which is the main user interface class of the ComPWA expert system

In [2]:
initial_state = [("J/psi", [-1, 1])]

final_state = [("eta", [0]), ("p", [-1/2, 1/2]), ("pbar", [-1/2, 1/2])]

tbd_manager = StateTransitionManager(initial_state, final_state,
                                     formalism_type='canonical-helicity',
                                     topology_building='isobar')
#tbd_manager
print(final_state)
#final_state = [("K_S0", [0]), ("sigma+", [-1/2, 1/2]), ("pbar", [-1/2, 1/2])]
#tbd_manager.add_final_state_grouping(final_state)

[('eta', [0]), ('p', [-0.5, 0.5]), ('pbar', [-0.5, 0.5])]


#### 1.2. Preparation
Create all topology graphs using the isobar model (two-body decays).

Also initialize the graphs with the initial and final state. Remember that each interaction node defines their own set of conservation laws. The `StateTransitionManager` (STM) defines three interaction types:

| Interaction | Strength |
| --- | --- |
| strong | $60$ |
| electromagnetic (EM) | $1$ |
| weak | $10^{-4}$ |

Be default all three are used in the preparation stage. `prepare_graphs()` of the STM generates graphs with all possible combinations of interaction nodes. An overall interaction strength is assigned to each graph, and they are grouped according to this strength.

#### 1.3. Finding solutions
If you are happy with the automatic settings generated by the StateTransitionManager, just go directly to the solving!

In [3]:
tbd_manager.allowed_intermediate_particles = ["N(1650)+","N(1650)-"]
#tbd_manager.set_allowed_interaction_types(
#    [InteractionTypes.Strong, InteractionTypes.EM, InteractionTypes.Weak])
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)
print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)
if len(solutions)<1:
    raise ValueError("Number of Solutions is to low")

found 192 solutions!
intermediate states:
{'N(1650)+', 'N(1650)-'}


In [4]:
xml_generator = CanonicalAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file('ModelPPbarEta.xml')